In [373]:
import numpy as np
from keras.models import Model
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import vgg16
from PIL import Image

In [410]:
#constants
BATCH_SIZE = 4
NCPF = 32 #number of classes per file
TRAIN_SAMPLES = 16
VAL_SAMPLES = 4

STARTING_CLASS = 0*NCPF #change to the class we left off from

DATA_DIR = '../data/'
TYPE = 'block4_pool'
BIN_DIR = '../data/cal_predictions/{0}/'.format(TYPE)

In [411]:
#create model
vgg_model = vgg16.VGG16( weights='imagenet', include_top=True ) #downloads prebuilt
for layer in vgg_model.layers:
    layer.trainable = False
if TYPE == 'no_norm' or TYPE == 'norm':
    vgg_out = vgg_model.layers[-2].output
else:
    vgg_out = vgg_model.get_layer(TYPE)
    vgg_out = Flatten()(vgg_model.get_layer(TYPE).output)
model = Model( input=vgg_model.input, output=vgg_out )
model.compile( loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"] )

In [412]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_45 (InputLayer)            (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_45[0][0]                   
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [390]:
datagen = ImageDataGenerator(samplewise_center=False, samplewise_std_normalization=False)#standardizes per sample

In [391]:
#create a generator for the training samples starting from the class we left off from
generator = datagen.flow_from_directory(
            DATA_DIR + 'train',
            target_size=(224, 224),
            batch_size=BATCH_SIZE,
            class_mode=None,
            shuffle=False)
for _ in range(STARTING_CLASS*TRAIN_SAMPLES//BATCH_SIZE): #skip up to the starting class
    generator.next() 

Found 4096 images belonging to 256 classes.


In [392]:
#Image.fromarray(np.uint8(generator.next()[0]))

In [393]:
features = model.predict_generator(generator, TRAIN_SAMPLES*NCPF) #push through vgg_model

In [394]:
np.save(open(BIN_DIR + 'train/{0}_{1}.npy'.format(STARTING_CLASS,STARTING_CLASS+NCPF), 'wb'), features) #save features

In [395]:
#create a generator for the validation samples starting from the class we left off from
generator = datagen.flow_from_directory(
            DATA_DIR + 'validation',
            target_size=(224, 224),
            batch_size=BATCH_SIZE,
            class_mode=None,
            shuffle=False)
for _ in range(STARTING_CLASS*VAL_SAMPLES//BATCH_SIZE): #skip up to the starting class
    generator.next()

Found 1024 images belonging to 256 classes.


In [396]:
features = model.predict_generator(generator, VAL_SAMPLES*NCPF) #push through vgg_model

In [397]:
np.save(open(BIN_DIR + 'validation/{0}_{1}.npy'.format(STARTING_CLASS,STARTING_CLASS+NCPF), 'wb'), features) #save features